<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Masonry_AI_Researcher_Mohd_Kaif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install required libraries
!pip install -qU langchain langchain-groq langchain-community crawl4ai requests beautifulsoup4 googlesearch-python nest-asyncio request

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request


In [9]:
import os
import asyncio
import logging
import json
import re
from typing import List, Dict, Any
from datetime import datetime
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from crawl4ai import AsyncWebCrawler
from bs4 import BeautifulSoup
from googlesearch import search
import nest_asyncio

# Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set Groq API key (replace with your key or use Colab secrets)
os.environ["GROQ_API_KEY"] = "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"

In [10]:
# Initialize chat history store
chat_history_store = {}

def get_chat_history(session_id: str) -> ChatMessageHistory:
    """Retrieve or create chat history for a session."""
    if session_id not in chat_history_store:
        chat_history_store[session_id] = ChatMessageHistory()
    return chat_history_store[session_id]

# Initialize Groq LLMs
def init_llms() -> Dict[str, ChatGroq]:
    """Initialize Groq LLMs for summarization and analysis."""
    return {
        'summary': ChatGroq(model_name="llama3-70b-8192", temperature=0.7, api_key=os.environ.get("GROQ_API_KEY")),
        'analysis': ChatGroq(model_name="mixtral-8x7b-32768", temperature=0.5, api_key=os.environ.get("GROQ_API_KEY"))
    }

In [11]:
# Define LangChain tools
@tool
def search_web(query: str) -> List[Dict[str, str]]:
    """Perform a web search and return results."""
    logger.info(f"Searching web for: {query}")
    results = []
    try:
        for url in search(query, num_results=5):
            results.append({'url': url, 'title': '', 'snippet': ''})
        return results[:5]
    except Exception as e:
        logger.error(f"Search error: {str(e)}")
        return []

@tool
async def crawl_url(url: str) -> Dict[str, Any]:
    """Crawl a URL using Crawl4AI and return extracted content."""
    logger.info(f"Crawling URL: {url}")
    try:
        async with AsyncWebCrawler() as crawler:
            result = await crawler.arun(
                url=url,
                params={
                    "return_format": "markdown",
                    "proxy_enabled": False,
                    "request_timeout": 30,
                    "limit": 1
                }
            )
            if result.success:
                return {
                    'url': url,
                    'title': result.title or url,
                    'content': result.markdown[:2000],
                    'success': True
                }
    except Exception as e:
        logger.error(f"Error crawling {url}: {str(e)}")
    return {'url': url, 'title': '', 'content': '', 'success': False}

@tool
def extract_content_fallback(url: str) -> Dict[str, Any]:
    """Fallback content extraction using BeautifulSoup."""
    logger.info(f"Extracting content (fallback) from: {url}")
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text(separator=' ', strip=True)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)[:2000]
        title = soup.title.string if soup.title else url
        return {'url': url, 'title': title, 'content': text, 'success': True}
    except Exception as e:
        logger.error(f"Error extracting content from {url}: {str(e)}")
        return {'url': url, 'title': '', 'content': '', 'success': False}

In [12]:
# Define LangChain tools
@tool
def search_web(query: str) -> List[Dict[str, str]]:
    """Perform a web search and return results."""
    logger.info(f"Searching web for: {query}")
    results = []
    try:
        for url in search(query, num_results=5):
            results.append({'url': url, 'title': '', 'snippet': ''})
        return results[:5]
    except Exception as e:
        logger.error(f"Search error: {str(e)}")
        return []

@tool
async def crawl_url(url: str) -> Dict[str, Any]:
    """Crawl a URL using Crawl4AI and return extracted content."""
    logger.info(f"Crawling URL: {url}")
    try:
        async with AsyncWebCrawler() as crawler:
            result = await crawler.arun(
                url=url,
                params={
                    "return_format": "markdown",
                    "proxy_enabled": False,
                    "request_timeout": 30,
                    "limit": 1
                }
            )
            if result.success:
                return {
                    'url': url,
                    'title': result.title or url,
                    'content': result.markdown[:2000],
                    'success': True
                }
    except Exception as e:
        logger.error(f"Error crawling {url}: {str(e)}")
    return {'url': url, 'title': '', 'content': '', 'success': False}

@tool
def extract_content_fallback(url: str) -> Dict[str, Any]:
    """Fallback content extraction using BeautifulSoup."""
    logger.info(f"Extracting content (fallback) from: {url}")
    session = init_session()
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text(separator=' ', strip=True)
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)[:2000]
        title = soup.title.string if soup.title else url
        return {'url': url, 'title': title, 'content': text, 'success': True}
    except Exception as e:
        logger.error(f"Error extracting content from {url}: {str(e)}")
        return {'url': url, 'title': '', 'content': '', 'success': False}

In [13]:
# Main Research Pipeline
async def research(query: str, session_id: str = "default") -> Dict[str, Any]:
    """Main research pipeline."""
    logger.info(f"Starting research for query: {query}")

    # Initialize LLMs and agent
    llms = init_llms()
    agent_executor = init_agent(llms['analysis'])

    # Step 1: Analyze query
    query_analysis = analyze_query(query)

    # Step 2: Execute agent for search and crawling
    try:
        agent_result = await agent_executor.ainvoke({"input": query_analysis['search_terms']})
        search_results = agent_result.get('output', [])
        if isinstance(search_results, str):
            search_results = [{'url': url, 'title': '', 'snippet': ''} for url in search_results.split()]
    except Exception as e:
        logger.error(f"Agent execution error: {str(e)}")
        search_results = search_web.invoke(query_analysis['search_terms'])

    # Step 3: Crawl content
    contents = []
    for result in search_results[:5]:
        content = await crawl_url.invoke(result['url'])
        if not content['success']:
            content = extract_content_fallback.invoke(result['url'])
        contents.append(content)

    # Step 4: Analyze content
    analyzed_contents = analyze_content(contents, query)

    # Step 5: Synthesize information
    report = await synthesize_information(analyzed_contents, query, session_id)

    return report

# Save Report
def save_report(report: Dict[str, Any], filename: str = "research_report.json"):
    """Save the research report to a JSON file."""
    with open(filename, 'w') as f:
        json.dump(report, f, indent=2)
    print(f"Report saved to {filename}")

# Test the Agent
async def test_research_agent():
    """Test the research pipeline with sample queries."""
    test_queries = [
        "What is the capital of France?",
        "Recent news about AI developments"
    ]
    for query in test_queries:
        print(f"\nTesting query: {query}")
        result = await research(query, session_id=f"test_{query}")
        print(json.dumps(result, indent=2))
        save_report(result, f"research_report_{query.replace(' ', '_')}.json")

# Run tests in Colab
if __name__ == "__main__":
    asyncio.run(test_research_agent())


Testing query: What is the capital of France?


NameError: name 'init_agent' is not defined